# 1. Preliminaries

In [2]:
##!pip3 install qutip
import numpy as np
import qutip as qp
import scipy as si
import pandas as pd
from random import random, choice, randint
##from multiprocessing import Pool
##from concurrent.futures import ThreadPoolExecutor

# 2. Gell_Mann matrix generator function

this function will generate the gellmann matrix of dimension d.

the function "gellmann(j,k,d)" will generate the jth_kth gellmann matrix of dimension d. so for every d dimension we have d^2 gellmann matrix. one of them is I, so the number of gellmann matrix that is not trivial is (d^2 - 1).

In [3]:
"""Generate generalized Gell-Mann matrices.
  .. module:: gellmann.py
     :synopsis: Generate generalized Gell-Mann matrices
  .. moduleauthor:: Jonathan Gross <jarthurgross@gmail.com>
"""
from itertools import product

def gellmann(j, k, d):
    r"""Returns a generalized Gell-Mann matrix of dimension d.
    
    According to the convention in *Bloch Vectors for Qubits* by Bertlmann and
    Krammer (2008), returns :math:`\Lambda^j` for :math:`1\leq j=k\leq d-1`,
    :math:`\Lambda^{kj}_s` for :math:`1\leq k<j\leq d`, :math:`\Lambda^{jk}_a`
    for :math:`1\leq j<k\leq d`, and :math:`I` for :math:`j=k=d`.
    Parameters
    ----------
    j : positive integer
        Index for generalized Gell-Mann matrix
    k : positive integer
        Index for generalized Gell-Mann matrix
    d : positive integer
        Dimension of the generalized Gell-Mann matrix
    Returns
    -------
    numpy.array
        A genereralized Gell-Mann matrix.
    """

    if j > k:
        gjkd = np.zeros((d, d), dtype=np.complex128)
        gjkd[j - 1][k - 1] = 1
        gjkd[k - 1][j - 1] = 1
    elif k > j:
        gjkd = np.zeros((d, d), dtype=np.complex128)
        gjkd[j - 1][k - 1] = -1.j
        gjkd[k - 1][j - 1] = 1.j
    elif j == k and j < d:
        gjkd = np.sqrt(2/(j*(j + 1)))*np.diag([1 + 0.j if n <= j
                                               else (-j + 0.j if n == (j + 1)
                                                     else 0 + 0.j)
                                               for n in range(1, d + 1)])
    else:
        gjkd = np.diag([1 + 0.j for n in range(1, d + 1)])

    return gjkd

def get_basis(d):
    r"""Return a basis of operators.
    
    The basis is made up of orthogonal Hermitian operators on a Hilbert space
    of dimension d, with the identity element in the last place.
    Parameters
    ----------
    d : int
        The dimension of the Hilbert space.
    Returns
    -------
    list of numpy.array
        The basis of operators.
    """
    return [gellmann(j, k, d) for j, k in product(range(1, d + 1), repeat=2)]

in this cell the gellmann matrix of 3 dimension will generate and they will save in a dictionary and labeled them by numbers 0,1,...,8 .

In [4]:
def gellmann3():
    gellmann3_basis = {}

    k=0
    for i in range(1,4):
        for j in range(1,4):
            gellmann3_basis.update({"G{}".format(k):gellmann(i,j,3)})
            k+=1
        
    return gellmann3_basis

# 3. Coefficients of gellmann's expansion

at this cell a random density matrix will generate by qutip library.

for calculating the coefficients of gellmann expansion we do this iteration:
1. calculate the tensor product of gellmann matrices for every coefficient
2. calculate the dot product of "ro" and the "gellmann's tensor product"
3. calculate the trace of this matrix and divide it into 4
4. append it to our index list

In [5]:
def coef_of_gellmann(ro):

    coef = {}  ## the coefficient's dictionary (suitable format for saving as a data)
    gellmann3_basis = gellmann3() ## the gellmann matrix for a 3 dimension system
    
    for i in range(9):
        for j in range(9):
            tensorP = np.kron(gellmann3_basis["G{}".format(i)],gellmann3_basis["G{}".format(j)]) ## Tensor product of gelllmann matrices
            c = np.trace(np.dot(ro,tensorP))/4   ## dot product and trace
            c = c.real       ## all of gellmann coefs is a real
            c = round(c,8)  
            coef.update({"a{}{}".format(i,j):c})  ## update the coef dict by a[i][j] and its gellmann coef for g[i][j]

    index = np.array(coef)  ## change list to np.array() object
    return coef

this cell is just a test of this algotithm in 3 dimension bipatite systems.

In [6]:
N = 3

ro = qp.rand_dm(N*N,0.5,dims=[[N,N], [N,N]])
coef_of_gellmann(ro)

{'a00': -0.0180817,
 'a01': -0.01034997,
 'a02': -0.01499394,
 'a03': -0.00046424,
 'a04': -0.00893252,
 'a05': 0.01692853,
 'a06': 0.00829214,
 'a07': 0.01601582,
 'a08': 0.00202866,
 'a10': 0.02149436,
 'a11': -0.00804138,
 'a12': 0.00413451,
 'a13': -0.00787622,
 'a14': 0.00626229,
 'a15': -0.01120844,
 'a16': -0.00442607,
 'a17': -0.00564804,
 'a18': 0.03659789,
 'a20': -0.05208995,
 'a21': -0.00704855,
 'a22': 0.01457389,
 'a23': 0.01367472,
 'a24': -0.00031228,
 'a25': 0.01794845,
 'a26': -0.01113653,
 'a27': 0.00836012,
 'a28': 0.05279191,
 'a30': -0.03467806,
 'a31': 0.00175349,
 'a32': -0.01735262,
 'a33': -0.00387669,
 'a34': -0.01771898,
 'a35': 0.00564804,
 'a36': -0.01337906,
 'a37': -0.01120844,
 'a38': -0.02384558,
 'a40': 0.02930584,
 'a41': -0.01453152,
 'a42': 0.03617178,
 'a43': -0.01187914,
 'a44': 0.0164995,
 'a45': -0.00702485,
 'a46': -0.00577702,
 'a47': 2.494e-05,
 'a48': -0.0368276,
 'a50': -0.00335873,
 'a51': 0.00290104,
 'a52': 0.01912693,
 'a53': 0.0088154

# 4. Generate entangled states (PPT)

In this cell we define a PPT funcion that check positive partial transpose.

In [7]:
def PPT(ro):
    """ positive partial transpose 
    input : density matrix
    out put:list of  coefficient of gellmann matrices and detect entangeled states
    """

    ro_pt = qp.partial_transpose(ro,[0,1])  ## partial transpose in subsystem 2
    eig = ro_pt.eigenstates()   ## calculate the eigenvalues and eigenstates
    eigv = [round(i,8) for i in eig[0]] 
    
    result = coef_of_gellmann(ro)
    
    if (eigv[0]<0 ):
            result.update({"label":1})  ## the entangled states that labeled by 1
            return result
    else :
            result.update({"label":2})  ## the unknown states that labeled by 2
            return result


# 5. Generate separable states (Convex combination)

in this cell we define two function that:
1. Convex_Combination: this function is give a matrix list and return a convex combination of this matrices
2. generate_ran_den_list: this function is generate a list of random density matrices

In [8]:
def Convex_Combination(matrix_list, index=10):
    
    coef = np.random.dirichlet(np.ones(index),size=1)   ## generating list of numbers that is positive and its sum is 1 with length equal to 'index'
    
    seprable_state = qp.Qobj()    ## build an empty Qobj for cycle
    for i in range(index):
        a = choice(matrix_list)   ## choice a density matrix randomly
        b = choice(matrix_list)   ## choice a density matrix randomly
        
        tensorP = np.kron(a,b) ## Tensor product of density matrices
        tensorP *= round(coef[0][i],9)  ## product of tensorP and a coef that come from the coef list
        
        seprable_state += tensorP  ## at the last its a convex combination of tensor product of density matrices
    
    return seprable_state


def generate_ran_den_list(dim, length= 100):
    
    result = []
    for i in range(length):
        density = round(random(),4)     ## for build a random number between 0 and 1

        rand = qp.rand_dm(dim, density)   ## generate a random density matrix with dimension N
        result.append(rand)        ## appending this random density matrix to ran_den_mat list    
    
    return result

# 6. Generate data

in this cell we build 500 000 random states and labeled that. after that we build a data frame for saving and using in future.

In [9]:
#%%time

states={} #states
dim = 3   ## dimension of a system
R = 250000  ## the number of entangled states

##generate the entangled states
for i in range (1, R+1):
    density = round(random(),4)     ## for build a random number between 0 and 1
    
    ro = qp.rand_dm(dim*dim, density, dims=[[dim,dim], [dim,dim]])     #generating random density matrix with a random density
    states.update({"{}".format(i):PPT(ro)})     # update the states by its number and label and gellmann coefs


##generate the separable states
ran_den_mat = generate_ran_den_list(dim, 10000) ## a list of random density matrix for a system

for i in range(R+1, 2*R+1):
    index = randint(10,100)   ## build a number between 10 and 100 for using in function
    sep = Convex_Combination(ran_den_mat, index)  ## generate a random seprable density matrix
    
    sep_coef = coef_of_gellmann(sep)   ## calculate the gellmann coefs
    sep_coef.update({"label":0})   ## labeled seprable states by 0
    
    states.update({"{}".format(i):sep_coef})  ## update sep_states
    
    
df = pd.DataFrame(data=states).T  # build a data frame by pandas
df
#df.to_csv('data_of_states.csv.zip')

Wall time: 43.6 s


in this cell we save our data frame as a zip file

In [19]:
df.to_csv('data_of_states.csv.zip')
##df.to_csv('data_of_states.csv')